# Environment

In [1]:
! pip install --quiet langchain langchain_cohere langchain-openai tiktoken langchainhub chromadb langgraph mem0ai openai
! pip install --quiet -r ./requirements.txt

In [1]:
import os
import time
import numpy as np
import json
import embedding_model_setup
from langchain_cohere import CohereEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.document_loaders import PyPDFDirectoryLoader
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere
from langchain_community.vectorstores import FAISS
from langchain_core.pydantic_v1 import BaseModel, Field

# Load environment variables from .env file
load_dotenv()

llm_groq=ChatGroq(groq_api_key=os.getenv("GROQ_API_KEY"),
             model_name="Llama3-8b-8192")
llm_cohere=ChatCohere(model="command-r", temperature=0)

# Load the SentenceTransformer model for embeddings
embedding_model_name = "FinLang/finance-embeddings-investopedia"
embedding_model = embedding_model_setup.SentenceTransformerEmbeddings(embedding_model_name)

# Loading the Cohere embedding model
embedding_model_cohere = CohereEmbeddings()

c:\Users\ASUS\Desktop\Stuff\Prototype for Hackathon\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


## Index

In [2]:
def vector_embedding():
    """
    Load documents, split them into chunks, and create embeddings for each chunk.
    Then, upsert these embeddings into the FAISS index.
    """
    # Load documents from the specified directory
    loader = PyPDFDirectoryLoader("./DATA")
    docs = loader.load()

    # Split documents into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    final_documents = text_splitter.split_documents(docs[:20])  # Limit to the first 20 documents

    docsearch_STHF = FAISS.from_documents(
        documents=final_documents,
        embedding=embedding_model, 
    )
    docsearch_Cohere = FAISS.from_documents(
        documents=final_documents,
        embedding=embedding_model_cohere, 
    )
    
    return docsearch_STHF, docsearch_Cohere


sthf_knowledge, cohere_knowledge = vector_embedding()
sthf_retriever, cohere_retriever = sthf_knowledge.as_retriever(), cohere_knowledge.as_retriever() 

## LLMs

We use a router to pick between tools. 
 
Cohere model decides which tool(s) to call, as well as the how to query them.

In [3]:
from langchain_community.document_loaders import PyPDFLoader

class CategorizeQuestion(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    category: str = Field(
        description="Category belongs to either 'Can be Found in the Annual Report' or 'Typically to be Searched on the Internet' or 'Can be answered without reference'"
    )

def VE_for_categorization(pdf_file_path):
    """
    Load a single PDF file, split it into chunks, and create embeddings for each chunk.
    Then, upsert these embeddings into the FAISS index.
    """
    # Load the single PDF file
    loader = PyPDFLoader(pdf_file_path)
    docs = loader.load()

    # Split documents into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    final_documents = text_splitter.split_documents(docs)

    # Create and return a FAISS vector store
    docsearch = FAISS.from_documents(
        documents=final_documents,
        embedding=embedding_model_cohere,
    )
    
    return docsearch


pdf_file_path = "./Questions.pdf"  # Specify the path to your single PDF file
print("Starting the vector embedding process for categorization...")
categorize_knowledge = VE_for_categorization(pdf_file_path)  # Generate embeddings and populate FAISS
categorize_retriever = categorize_knowledge.as_retriever()
print("Vector Store DB is ready.")

preamble = """You are an expert at categorizing a user question into either "Can be found in Annual Report" or "Typically Searched on the Internet" or "Can be answered without reference".
    Categorize the user question into these categories, just generate the name of the category and nothing else."""

# Define the categorization prompt
categorization_prompt = ChatPromptTemplate.from_messages(
    [('system', preamble),('human',"""
    <context>
    {context}
    </context>
    Question: {input}""")]
)


Starting the vector embedding process for categorization...
Vector Store DB is ready.


In [4]:
question = "What can you do for me ?"
# Create a retrieval chain for categorization
llm=ChatGroq(groq_api_key=os.getenv("GROQ_API_KEY"),
             model_name="Llama3-8b-8192")
llm_structured_output = llm.with_structured_output(CategorizeQuestion)

# document_chain = create_stuff_documents_chain(llm_structured_output, categorization_prompt)
# categorize_retrieval_chain = create_retrieval_chain(categorize_retriever, document_chain)
# print(retriever.invoke(question))
# Generate the category for the question
# response = categorize_retrieval_chain.invoke({'input': question})

categorization_chain = categorization_prompt | llm_structured_output
response = categorization_chain.invoke({"context": categorize_retriever, "input": question})

In [5]:
response

CategorizeQuestion(category='Can be answered without reference')

In [6]:
### Router
from langchain_cohere import ChatCohere
from langchain_core.prompts import ChatPromptTemplate


# Data model
class web_search(BaseModel):
    query: str = Field(description="The query to use when searching the internet.")


class vectorstore(BaseModel):
    query: str = Field(description="The query to use when searching the vectorstore.")

class fallback(BaseModel):
    query: str = Field(description="The query to use when answering without reference.")


# Preamble
preamble = """You are an expert at routing a phrase to a vectorstore or web search or fallback.
Use the vectorstore for phrase involving similar to "Can be Found in the Annual Report", use web-search for "Typically to be Searched on the Internet". Otherwise use fallback for everything else"""

# LLM with tool use and preamble
llm = ChatCohere(model="command-r", temperature=0)
structured_llm_router = llm.bind_tools(
    tools=[web_search, vectorstore, fallback], preamble=preamble
)

# Prompt
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{phrase}"),
    ]
)

phrase_router = route_prompt | structured_llm_router
response = phrase_router.invoke({"phrase": "Can be found on the Annual Report"})
print(response.response_metadata["tool_calls"])
response = phrase_router.invoke({"phrase": "Typically searched on the internet"})
print(response.response_metadata["tool_calls"])
response = phrase_router.invoke({"phrase": "must be present in the Annual Report"})
print(response.response_metadata["tool_calls"])
response = phrase_router.invoke({"phrase": "can be answered without reference"})
print(response.response_metadata["tool_calls"])
print("tool_calls" in response.response_metadata)

[{'id': 'e3fa67ba0adb44b3814b2971593410e4', 'function': {'name': 'vectorstore', 'arguments': '{"query": "Can be found on the Annual Report"}'}, 'type': 'function'}]
[{'id': 'b2b2edd179114c63ac4ec884636c3bfa', 'function': {'name': 'web_search', 'arguments': '{"query": "Typically searched on the internet"}'}, 'type': 'function'}]
[{'id': 'd1723d6a674940e29d2cef1e1bd5cd44', 'function': {'name': 'vectorstore', 'arguments': '{"query": "must be present in the Annual Report"}'}, 'type': 'function'}]
[{'id': '847c604c71b14ba1b8cf84bab5d25983', 'function': {'name': 'fallback', 'arguments': '{"query": "can be answered without reference"}'}, 'type': 'function'}]
True


In [7]:
### Retrieval Grader

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

# Prompt
preamble = """You are a grader assessing relevance of a retrieved document to a user question. \n
If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

# LLM with function call
llm = ChatCohere(model="command-r", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments, preamble=preamble)

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "Tell me about the company and its business as detailed as possible"
docs = sthf_retriever.invoke(question)
doc_txt = docs[1].page_content
print(doc_txt)
response = retrieval_grader.invoke({"question": question, "document": doc_txt})
print(response)

share, taking the total dividend for the year to ₹73 per share. 
The company also completed its fifth successful buyback returning ₹17,000 crore to shareholders, wherein the buyback 
process was completed in record time of 63 days. The total shareholder payout  for the year was ₹47,445 crore.
Recognized as a Global Top Employer  by the Top Employers 
Institute for the ninth consecutive year, for TCS’ pioneering employee engagement and talent development initiatives. TCS was also named a top employer in 32 countries and regions, 
including Europe, the UK, the Middle East, North America, Latin America, and South-East Asia.
Ranked the #1 IT service provider for customer satisfaction 
in Europe in an independent survey of over 2,000 CXOs of 
the continent’s top IT spenders by Whitelane Research, for the 11
th consecutive time.  The study also revealed that TCS
binary_score='yes'


Generate

In [ ]:
### Generate

from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser

# Preamble
preamble = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as detailed according to the level of expertise of the user, in bullet points"""

# Prompt
def prompt(x):
    return ChatPromptTemplate.from_messages(
        [
            ("system", f"{preamble}"), 
            HumanMessage(
                f"Question: {x['question']} \nAnswer: ",
                additional_kwargs={"documents": x["documents"]},
            )
        ]
    )


# # Chain
# llm = ChatCohere(model="command-r", temperature=0)
# openai_rag_chain = prompt | llm | StrOutputParser()

# # Run
# generation = openai_rag_chain.invoke({"documents": docs, "question": question})
# print(generation)

# ! pip install openai

import os
from openai import OpenAI

open_ai_client = OpenAI(
    api_key = os.getenv("OPENAI_API_KEY"),
)

completion = open_ai_client.chat.completions.create( # Change the method
  model = "gpt-4o-mini",
  messages = [ # Change the prompt parameter to messages parameter
    {"role": "system", "content": f"{preamble}"},
    {"role": "user", "content": f"""
        user expertise: expert
        Context: {docs}
        Question: {question}
    """},
  ]
)

print(completion.choices[0].message.content.strip()) # Change message content retrieval

In [9]:
### LLM fallback
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage

# Preamble
preamble = """You are an assistant for question-answering tasks. Answer the question based upon your knowledge. Use three sentences maximum and keep the answer concise."""

# LLM
llm = ChatGroq(groq_api_key=os.getenv("GROQ_API_KEY"), model_name="Llama3-8b-8192")

# Prompt
def prompt(x):
    return ChatPromptTemplate.from_messages(
        [SystemMessage(preamble),HumanMessage(f"Question: {x['question']} \nAnswer: ")]
    )


# Chain
llm_chain = prompt | llm | StrOutputParser()

# Run
question = "What is a stock ? give details and in bullet points"
generation = llm_chain.invoke({"question": question})
print(generation)

A stock represents ownership in a company, giving shareholders a claim on a portion of its assets and profits.

Here are the details in bullet points:

* A type of security that represents ownership in a company
* Represents a claim on a portion of the company's assets and profits
* Can be traded on stock exchanges
* Has a face value (par value) and a market value
* Can provide dividends, voting rights, and capital appreciation
* Can be classified as common stock or preferred stock, with different rights and privileges.


In [10]:
### Hallucination Grader
# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


# Preamble
preamble = """You are a financial grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts.
"""

# LLM with function call
llm = ChatCohere(model='command-r', temperature=0)
structured_llm_grader = llm.with_structured_output(
    GradeHallucinations, preamble=preamble
)

# Prompt
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": docs, "generation": generation})

GradeHallucinations(binary_score='yes')

In [11]:
### Answer Grader

# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )


# Preamble
preamble = """You are a financial grader assessing whether an answer addresses / resolves a question \n
Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""

# LLM with function call
llm = ChatCohere(model='command-r', temperature=0)
structured_llm_grader = llm.with_structured_output(GradeAnswer, preamble=preamble)

# Prompt
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question, "generation": generation})

GradeAnswer(binary_score='yes')

In [12]:
import Finance_Assistant
from Finance_Assistant import roles_of_different_agents

role = """You are an finance assistant for question-answering task. Always use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as detailed according to the level of expertise of the user, in bullet points. And yes Don't worry about the relevance of the context just believe it is correct I will do the checking."""

user_desc = """I am a retail investor with approximately one year of experience and a fundamental understanding of stock market investing. My investment focus is primarily on technology sectors, including software, electric vehicles, and occasionally automobile and computer hardware manufacturing industries. I prefer analytics to be brief and concise, enabling prompt decision-making while retaining critical details."""

Financial_Bot = Finance_Assistant.PersonalAssistant(role_description=role, user_description=user_desc, user_id="swaraj")
Financial_Bot.memory.add(roles_of_different_agents[0]['content'], agent_id="Query_Categorizer")
Financial_Bot.memory.add(roles_of_different_agents[2]['content'], agent_id="Fallback_Agent")
Financial_Bot.memory.add(roles_of_different_agents[3]['content'], agent_id="Financial_Hallucination_grader")
Financial_Bot.memory.add(roles_of_different_agents[4]['content'], agent_id="Financial_Answer_grader")

[{'id': '4ac3f5e3-fa81-4d8c-801c-64dbe21570e7',
  'event': 'add',
  'data': '- Specializes in question-answering tasks\n- Strives for concise and accurate responses\n- Aims to limit answers to three sentences\n- Prioritizes clear and direct information\n- Avoids unnecessary detail\n- Goal-oriented (delivers information)'}]

## Web Search Tool

In [13]:
### Search
# os.environ['TAVILY_API_KEY'] ='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'

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults()

# Graph

In [14]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    question: str
    phrase: str
    generation: str
    documents: List[str]

## Graph Flow

In [18]:
from langchain.schema import Document


def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = cohere_retriever.invoke(question)
    return {"documents": documents, "question": question}


def llm_fallback(state):
    print("---LLM Fallback---")
    question = state["question"]
    generation = llm_chain.invoke({"question": question})

    # Storing in Agent memory and master memory
    Financial_Bot.memory.add(f"Question By User: {question}", agent_id="Fallback_Agent")
    Financial_Bot.memory.add(f"""Question asked: {question}, Answer Generated: {generation}""", user_id="swaraj")

    return {"question": question, "generation": generation}


def generate(state):
    """
    Generate answer using the vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    if not isinstance(documents, list):
        documents = [documents]
    
    documents_text = """"""
    for d in documents:
        documents_text += f"{d.page_content}\n\n"
    
    print(documents_text)

        
    # RAG generation and Saving in memory
    generation = Financial_Bot.ask_question(question, documents_text)

    # print(f"GENERATION OUTPUT: {generation}")
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": web_results, "question": question}


### Edges ###
def categorize_question(state):
    print("---CATEGORIZE QUESTION---")
    question = state['question']

    response = categorization_chain.invoke({'context':categorize_retriever, 'input': question})
    phrase = response.category
    Financial_Bot.memory.add(f"Question Asked: {question}, Categorized as: {phrase}", agent_id="Query_Categorizer")

    return {"phrase": phrase}

def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    phrase = state["phrase"]
    source = phrase_router.invoke({"phrase": phrase})

    # Fallback to LLM or raise error if no decision
    if "tool_calls" not in source.additional_kwargs:
        print("---ROUTE QUESTION TO LLM---")
        return "llm_fallback"
    if len(source.additional_kwargs["tool_calls"]) == 0:
        raise "Router could not decide source"

    # Choose datasource
    datasource = source.additional_kwargs["tool_calls"][0]["function"]["name"]
    if datasource == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "web_search"
    elif datasource == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"
    else:
        print("---ROUTE QUESTION TO LLM---")
        return "vectorstore"


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, WEB SEARCH---")
        return "web_search"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    if score != None:
            grade = score.binary_score
    else:
        print("---RETURNED NONE: PROBLEM WHILE HALLUCINATION CHECKING---")
        grade = 'yes'

    Financial_Bot.memory.add(f"Question Asked: {question}\n Hallucinated: {grade}", agent_id="Financial_Hallucination_grader")

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        if score != None:
            grade = score.binary_score
        else:
            print("---RETURNED NONE: PROBLEM WHILE ANSWER GRADING--=-")
            grade = 'yes'

        Financial_Bot.memory.add(f"""Question Asked: {question}, Correctly Answered: {grade}""", agent_id="Financial_Answer_grader")

        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [19]:
import pprint

from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("categorize_question", categorize_question)
workflow.add_node("web_search", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve

workflow.add_node("grade_documents", grade_documents)  # grade documents

workflow.add_node("generate", generate)  # rag
workflow.add_node("llm_fallback", llm_fallback)  # llm

# Build graph
workflow.set_entry_point("categorize_question")
workflow.add_conditional_edges(
    "categorize_question",
    route_question,
    {
        "web_search": "web_search",
        "vectorstore": "retrieve",
        "llm_fallback": "llm_fallback",
    },
)
# workflow.add_edge("web_search", "generate")
# workflow.add_edge("retrieve", "grade_documents")
# workflow.add_edge("grade_documents", "generate")
# workflow.add_edge("generate", END)
# workflow.add_edge("llm_fallback", END)

################################################
workflow.add_edge("web_search", "generate")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "web_search": "web_search",
        "generate": "generate",
    },
)
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": 'generate',  # Hallucinations: re-generate
        "not useful": 'web_search',  # Fails to answer question: fall-back to web-search
        "useful": END,
    },
)
# workflow.add_edge("generate", END)
workflow.add_edge("llm_fallback", END)
################################################

# Compile
app = workflow.compile()

In [20]:
# Run
inputs = {
    "question": """
    Tell me the list of exactly 10 latest news related to Tesla motors
    """
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        if key == 'generate':
            pprint.pprint(f"Node '{key}': {value}")
        # Optional: print full state at each node
    pprint.pprint("\n---\n")

# # Final generation
pprint.pprint(key)
pprint.pprint(value["generation"])

---CATEGORIZE QUESTION---
---ROUTE QUESTION---
---ROUTE QUESTION TO WEB SEARCH---
'\n---\n'
---WEB SEARCH---
'\n---\n'
---GENERATE---
Tesla news: All the latest Tesla news and updates
Keep on top of all the latest Tesla news right here
Tesla is the biggest name in the electric vehicles world at the moment, having somewhat blazed a trail with all-electric cars despite being a rather new company compared to tradition car brands.  December 2: Tesla is offering a discount on Model 3 and Model Y cars, if you buy this year
Tesla has reduced the price of the Model 3 and Model Y by $3,750, provided you buy the car before the end of December. According to Tesla North, the owner has reduced the weight of the car by 500 or 600 pounds, and that likely played a huge part in the speeds the car achieved.
 June 15: Tesla Model S Plaid might have just set a new 1/4 mile record
The flagship Tesla Model S Plaid is already a speedy machine, with a 0-60 time of 1.99 seconds — which Tesla claims makes it th

In [21]:
print(value['generation'])

Here are the 10 latest news updates related to Tesla Motors:

1. **Discount on Model 3 and Model Y**: Tesla has reduced the price of the Model 3 and Model Y by $3,750 if bought before the end of December.

2. **Model S Plaid 1/4 Mile Speed Record**: The Tesla Model S Plaid reportedly set a new 1/4 mile record with a 0-60 time of 1.99 seconds.

3. **New User Interface**: Tesla introduced a new customizable user interface for the Model S and Model X, with expected improvements for the Model 3 and Model Y.

4. **Steam Integration in Model S and X**: The latest Model S and Model X feature gaming performance enhancements, including an AMD RDNA 2 GPU, similar to a PlayStation 5.

5. **Interior Space Improvements**: Enhancements to the interior of the Model S were announced, including more space in the second row.

6. **New Battery Pack for Model S Plaid**: A new battery pack was confirmed for the Model S Plaid, although further details were not disclosed.

7. **Delayed Delivery Estimates for

In [27]:
print(Financial_Bot.get_memories()) # master memory - stores all the convos

print(Financial_Bot.memory.get_all(agent_id="Query_Categorizer"))
print(Financial_Bot.memory.get_all(agent_id="Financial_Hallucination_grader"))
print(Financial_Bot.memory.get_all(agent_id="Financial_Answer_grader"))

['Tesla reduced the price of Model 3 and Model Y by $3,750 if bought before the end of December. Tesla Model S Plaid reportedly set a new 1/4 mile record with a 0-60 time of 1.99 seconds. Tesla introduced a new customizable user interface for Model S and Model X, with expected improvements for Model 3 and Model Y. The latest Model S and Model X feature gaming performance enhancements, including an AMD RDNA 2 GPU. Tesla announced enhancements to the interior of the Model S, including more space in the second row. Tesla confirmed a new battery pack for the Model S Plaid. Delivery estimates for the upgraded Model 3 Long Range have been pushed back to March-April 2024. Tesla announced that the Model Y achieved the title of the best-selling vehicle of any kind in just four years.']
[{'id': '20c0846b-064b-469e-bb47-fed5f3bf50a4', 'text': 'Interested in news related to Tesla Motors\nPrefers news in the form of a list\nWants exactly 10 news items\nTypically searches for information on the inte

In [154]:
# Run
inputs = {"question": "Tell me about the financial growth of the company i.e TCS"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        # pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
        pass
    # pprint.pprint("\n---\n")

# Final generation
pprint.pprint(value["generation"])

---CATEGORIZE QUESTION---
---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
---GENERATE---
book for FY 2024 came at an all-time high of US$ 42.7 billion  
supported by strong client relationships and engagement. Client metrics continue to exhibit healthy progress with strong client additions. Employee retention continues to be at benchmark levels in the industry. TCS has been selected as a Top Employer of Choice in 32 countries. TCS has retained its ranking as the second most valuable global IT services brand, valued at US$19.2 billion,  
an increase of US$2 billion  from last year.

Financial  Capital
TCS’ longevity is a testimony to the strength of our business model and our ability to reinvent ourselves in an ever-evolving technology 
landscape to sta